In [ ]:
#!pip install selenium
!pip install contractions
!pip install prettytable

In [ ]:
from selenium import webdriver
import time
import csv
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import re
import contractions
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import bigrams, trigrams
import random
import collections
from nltk.classify import NaiveBayesClassifier
from nltk.classify import DecisionTreeClassifier
from nltk.classify import MaxentClassifier
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC
from nltk.classify.util import accuracy
from nltk.metrics.scores import (accuracy, precision, recall, f_measure)
from prettytable import PrettyTable

In [ ]:
#creating a dictionary of words and assigning True values to them
def bag_of_words(words):
    return dict([(word, True) for word in words])

# return a list of bigrams
def bigramReturner (words):
    word_list = words
    bigramFeatureVector = []
    for item in nltk.bigrams(word_list):
        bigramFeatureVector.append(' '.join(item))
    return bigramFeatureVector

# return a list of trigrams
def trigramReturner (words):
    word_list = words
    trigramFeatureVector = []
    for item in nltk.trigrams(word_list):
        trigramFeatureVector.append(' '.join(item))
    return trigramFeatureVector

# splits data 70/30
def split_data(reviews_list, split=0.70):
    
    split_point = int(round(split*len(reviews_list),0))
    split_point_2 = len(reviews_list) - split_point
    
    train, test = reviews_list[:split_point], reviews_list[-split_point_2:]

    return train, test

In [ ]:
del rating_text
del review_text
del combined_ratings_reviews_list

In [ ]:
browser = webdriver.Chrome()

combined_ratings_reviews_list = []

url_part_1 = "https://www.amazon.com/Cellucor-Workout-Extract-Limeade-Servings/product-reviews/B00XFA544E/ref=cm_cr_arp_d_paging_btm_next_"
url_part_2 = "?pageNumber="

for a in range (1,61):
    
    url = url_part_1 + str(a) + url_part_2 + str(a)

    browser.get(url)

    # Give the page 3 sec to load
    time.sleep(4)
    
    html = browser.page_source

    soup = BeautifulSoup(html, "lxml")
    
    containers = soup.find_all('div', {'data-hook':'review'})
    
    for f in range (0, len(containers)):
        f_container = containers[f]
        
        title_text = f_container.find('a', {'data-hook':'review-title'}).find('span').text
        
        review_text = f_container.find('span', class_ = 'a-size-base review-text review-text-content').find('span').text
        
        rating_raw = int(f_container.find('span', class_ ="a-icon-alt").text[0])

        rating_text = ''

        if rating_raw == 5:
            rating_text = 'pos'
        elif rating_raw == 4:
            rating_text = 'pos'
        elif rating_raw == 3:
            rating_text = 'neg'
        elif rating_raw == 2:
            rating_text = 'neg'
        elif rating_raw == 1:
            rating_text = 'neg'
            
        title_review_combined = str(title_text) + ' ' + str(review_text)
            
        combined_ratings_reviews_list.append([title_review_combined, rating_text])
    
browser.close()

In [ ]:
print(len(combined_ratings_reviews_list))

In [ ]:
random.shuffle(combined_ratings_reviews_list)

In [ ]:
unigrams = []
bigrams = []
trigrams = []
ngrams = []

for d in range(0,len(combined_ratings_reviews_list)):
    record = combined_ratings_reviews_list[d]
    
    review = record[0]
    rating = record[1]
    
    
    lower_text = review.lower()
    digit_free_text = re.sub(r'\d+', '', lower_text)
    words = word_tokenize(digit_free_text)
    
    expanded_contractions_tokens = []
    
    for a in words:
        fixed_token = contractions.fix(a)
        fixed_token_list = fixed_token.split()
        for b in range(0, len(fixed_token_list)):
            expanded_contractions_tokens.append(fixed_token_list[b])
    
    stripped_words = [word.strip(string.punctuation) for word in expanded_contractions_tokens]

    further_stripped_words = [word.strip('…') for word in stripped_words]

    split_words = []
    
    for word in further_stripped_words:
        x = word.find('.')
        if x == -1:
            split_words.append(word)
        else:
            word1 = word[0:x]
            word2 = word[x+1:len(word)]
            split_words.append(word1)
            split_words.append(word2)

    further_split_words = []
    
    for word in split_words:
        x = word.find('.')
        if x == -1:
            further_split_words.append(word)
        else:
            word1 = word[0:x]
            word2 = word[x+1:len(word)]
            further_split_words.append(word1)
            further_split_words.append(word2)
    
    furthest_split_words = []
    
    for word in further_split_words:
        x = word.find('/')
        if x == -1:
            furthest_split_words.append(word)
        else:
            word1 = word[0:x]
            word2 = word[x+1:len(word)]
            furthest_split_words.append(word1)
            furthest_split_words.append(word2)
  
    punct_free_words = [word for word in furthest_split_words if word not in set(string.punctuation)]

    english_stops = set(stopwords.words('english'))
    
    stop_free_words = [word for word in punct_free_words if word not in english_stops]

    wordnet_lemmatizer = WordNetLemmatizer()

    lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in stop_free_words]
    
    characters_to_remove = ['...', "'s", '', 'le', 'u', '’', 'le', 'ca', 'st', 'nd', 'ft',
                            '”', "", ' ', '  ' 'th', 'e', 'p', 'wo', 'wg', 'd', 'gon', 'na',
                            '“', 'n', 'ê', 'pt', 'ot']
    
    review_list = [word for word in lemmatized_words if word not in characters_to_remove]
    
    bigrams_list = bigramReturner(review_list)

    trigrams_list = trigramReturner(review_list)

    ngrams_list = review_list + bigrams_list + trigrams_list

    unigrams_feature_set = bag_of_words(review_list)

    bigrams_feature_set = bag_of_words(bigrams_list)

    trigrams_feature_set = bag_of_words(trigrams_list)

    ngrams_feature_set = bag_of_words(ngrams_list)
    
    unigrams.append((unigrams_feature_set, rating))

    bigrams.append((bigrams_feature_set, rating))

    trigrams.append((trigrams_feature_set, rating))

    ngrams.append((ngrams_feature_set, rating))

In [ ]:
unigrams_training_set, unigrams_testing_set = split_data(unigrams)

bigrams_training_set, bigrams_testing_set = split_data(bigrams)

trigrams_training_set, trigrams_testing_set = split_data(trigrams)

ngrams_training_set, ngrams_testing_set = split_data(ngrams)

In [ ]:
# creates a naive bayes classifier with the unigrams_training_set as input
nb_unigrams_classifier = nltk.NaiveBayesClassifier.train(unigrams_training_set)

# calculates the accuracy of the unigrams naive bayes classifier
# as well as the precision, recall, and f-measure of both the 
# positive and negative classes
nb_unigrams_refsets = collections.defaultdict(set)
nb_unigrams_testsets = collections.defaultdict(set)
nb_unigrams_labels = []
nb_unigrams_tests = []
 
for i, (feats, label) in enumerate(unigrams_testing_set):
    nb_unigrams_refsets[label].add(i)
    nb_unigrams_observed = nb_unigrams_classifier.classify(feats)
    nb_unigrams_testsets[nb_unigrams_observed].add(i)
    nb_unigrams_labels.append(label)
    nb_unigrams_tests.append(nb_unigrams_observed)
    
nb_unigrams_accuracy = round(nltk.classify.accuracy(nb_unigrams_classifier, unigrams_testing_set),4)

nb_unigrams_pos_prec = round(precision(nb_unigrams_refsets['pos'], nb_unigrams_testsets['pos']),4)
nb_unigrams_pos_rec = round(recall(nb_unigrams_refsets['pos'], nb_unigrams_testsets['pos']),4)
nb_unigrams_pos_f_measure = round(f_measure(nb_unigrams_refsets['pos'], nb_unigrams_testsets['pos']),4)

nb_unigrams_neg_prec = round(precision(nb_unigrams_refsets['neg'], nb_unigrams_testsets['neg']),4)
nb_unigrams_neg_rec = round(recall(nb_unigrams_refsets['neg'], nb_unigrams_testsets['neg']),4)
nb_unigrams_neg_f_measure = round(f_measure(nb_unigrams_refsets['neg'], nb_unigrams_testsets['neg']),4)



# creates a naive bayes classifier with the bigrams_training_set as input
nb_bigrams_classifier = nltk.NaiveBayesClassifier.train(bigrams_training_set)


# calculates the accuracy of the bigrams naive bayes classifier
# as well as the precision, recall, and f-measure of both the 
# positive and negative classes
nb_bigrams_refsets = collections.defaultdict(set)
nb_bigrams_testsets = collections.defaultdict(set)
nb_bigrams_labels = []
nb_bigrams_tests = []
 
for i, (feats, label) in enumerate(bigrams_testing_set):
    nb_bigrams_refsets[label].add(i)
    nb_bigrams_observed = nb_bigrams_classifier.classify(feats)
    nb_bigrams_testsets[nb_bigrams_observed].add(i)
    nb_bigrams_labels.append(label)
    nb_bigrams_tests.append(nb_bigrams_observed)


nb_bigrams_accuracy = round(nltk.classify.accuracy(nb_bigrams_classifier, bigrams_testing_set),4)

nb_bigrams_pos_prec = round(precision(nb_bigrams_refsets['pos'], nb_bigrams_testsets['pos']),4)
nb_bigrams_pos_rec = round(recall(nb_bigrams_refsets['pos'], nb_bigrams_testsets['pos']),4)
nb_bigrams_pos_f_measure = round(f_measure(nb_bigrams_refsets['pos'], nb_bigrams_testsets['pos']),4)

nb_bigrams_neg_prec = round(precision(nb_bigrams_refsets['neg'], nb_bigrams_testsets['neg']),4)
nb_bigrams_neg_rec = round(recall(nb_bigrams_refsets['neg'], nb_bigrams_testsets['neg']),4)
nb_bigrams_neg_f_measure = round(f_measure(nb_bigrams_refsets['neg'], nb_bigrams_testsets['neg']),4)


# creates a naive bayes classifier with the trigrams_training_set as input
nb_trigrams_classifier = nltk.NaiveBayesClassifier.train(trigrams_training_set)


# calculates the accuracy of the trigrams naive bayes classifier
# as well as the precision, recall, and f-measure of both the 
# positive and negative classes
nb_trigrams_refsets = collections.defaultdict(set)
nb_trigrams_testsets = collections.defaultdict(set)
nb_trigrams_labels = []
nb_trigrams_tests = []
 
for i, (feats, label) in enumerate(trigrams_testing_set):
    nb_trigrams_refsets[label].add(i)
    nb_trigrams_observed = nb_trigrams_classifier.classify(feats)
    nb_trigrams_testsets[nb_trigrams_observed].add(i)
    nb_trigrams_labels.append(label)
    nb_trigrams_tests.append(nb_trigrams_observed)


nb_trigrams_accuracy = round(nltk.classify.accuracy(nb_trigrams_classifier, trigrams_testing_set),4)

nb_trigrams_pos_prec = round(precision(nb_trigrams_refsets['pos'], nb_trigrams_testsets['pos']),4)
nb_trigrams_pos_rec = round(recall(nb_trigrams_refsets['pos'], nb_trigrams_testsets['pos']),4)
nb_trigrams_pos_f_measure = round(f_measure(nb_trigrams_refsets['pos'], nb_trigrams_testsets['pos']),4)

nb_trigrams_neg_prec = precision(nb_trigrams_refsets['neg'], nb_trigrams_testsets['neg'])
nb_trigrams_neg_rec = round(recall(nb_trigrams_refsets['neg'], nb_trigrams_testsets['neg']),4)
nb_trigrams_neg_f_measure = f_measure(nb_trigrams_refsets['neg'], nb_trigrams_testsets['neg'])


# creates a naive bayes classifier with the ngrams_training_set as input
nb_ngrams_classifier = nltk.NaiveBayesClassifier.train(ngrams_training_set)


# calculates the accuracy of the ngrams naive bayes classifier
# as well as the precision, recall, and f-measure of both the 
# positive and negative classes
nb_ngrams_refsets = collections.defaultdict(set)
nb_ngrams_testsets = collections.defaultdict(set)
nb_ngrams_labels = []
nb_ngrams_tests = []
 
for i, (feats, label) in enumerate(ngrams_testing_set):
    nb_ngrams_refsets[label].add(i)
    nb_ngrams_observed = nb_ngrams_classifier.classify(feats)
    nb_ngrams_testsets[nb_ngrams_observed].add(i)
    nb_ngrams_labels.append(label)
    nb_ngrams_tests.append(nb_ngrams_observed)

nb_ngrams_accuracy = round(nltk.classify.accuracy(nb_ngrams_classifier, ngrams_testing_set),4)

nb_ngrams_pos_prec = round(precision(nb_ngrams_refsets['pos'], nb_ngrams_testsets['pos']),4)
nb_ngrams_pos_rec = round(recall(nb_ngrams_refsets['pos'], nb_ngrams_testsets['pos']),4)
nb_ngrams_pos_f_measure = round(f_measure(nb_ngrams_refsets['pos'], nb_ngrams_testsets['pos']),4)

nb_ngrams_neg_prec = round(precision(nb_ngrams_refsets['neg'], nb_ngrams_testsets['neg']),4)
nb_ngrams_neg_rec = round(recall(nb_ngrams_refsets['neg'], nb_ngrams_testsets['neg']),4)
nb_ngrams_neg_f_measure = round(f_measure(nb_ngrams_refsets['neg'], nb_ngrams_testsets['neg']),4)

In [ ]:
x = PrettyTable()

x.field_names = ["Features", "Metrics", "Naive Bayes"]

x.add_row([" ", "Accuracy", nb_unigrams_accuracy])
x.add_row([" ", "Pos Precision", nb_unigrams_pos_prec])
x.add_row([" ", "Pos Recall", nb_unigrams_pos_rec])
x.add_row(["Unigrams", "Pos F-Measure", nb_unigrams_pos_f_measure])
x.add_row([" ", "Neg Precision", nb_unigrams_neg_prec])
x.add_row([" ", "Neg Recall", nb_unigrams_neg_rec])
x.add_row(["________", "Neg F-Measure", nb_unigrams_neg_f_measure])

x.add_row([" ", "Accuracy", nb_bigrams_accuracy])
x.add_row([" ", "Pos Precision", nb_bigrams_pos_prec])
x.add_row([" ", "Pos Recall", nb_bigrams_pos_rec])
x.add_row(["Bigrams", "Pos F-Measure", nb_bigrams_pos_f_measure])
x.add_row([" ", "Neg Precision", nb_bigrams_neg_prec])
x.add_row([" ", "Neg Recall", nb_bigrams_neg_rec])
x.add_row(["________", "Neg F-Measure", nb_bigrams_neg_f_measure])

x.add_row([" ", "Accuracy", nb_trigrams_accuracy])
x.add_row([" ", "Pos Precision", nb_trigrams_pos_prec])
x.add_row([" ", "Pos Recall", nb_trigrams_pos_rec])
x.add_row(["Trigrams", "Pos F-Measure", nb_trigrams_pos_f_measure])
x.add_row([" ", "Neg Precision", nb_trigrams_neg_prec])
x.add_row([" ", "Neg Recall", nb_trigrams_neg_rec])
x.add_row(["________", "Neg F-Measure", nb_trigrams_neg_f_measure])

x.add_row([" ", "Accuracy", nb_ngrams_accuracy])
x.add_row([" ", "Pos Precision", nb_ngrams_pos_prec])
x.add_row([" ", "Pos Recall", nb_ngrams_pos_rec])
x.add_row(["N-grams", "Pos F-Measure", nb_ngrams_pos_f_measure])
x.add_row([" ", "Neg Precision", nb_ngrams_neg_prec])
x.add_row([" ", "Neg Recall", nb_ngrams_neg_rec])
x.add_row([" ", "Neg F-Measure", nb_ngrams_neg_f_measure])

x.align["Metrics"] = "l"
x.align["Naive Bayes"] = "l"

print(x)

In [ ]:
nb_unigrams_classifier.show_most_informative_features(100)

In [ ]:
nb_bigrams_classifier.show_most_informative_features(100)

In [ ]:
nb_trigrams_classifier.show_most_informative_features(100)

In [ ]:
nb_ngrams_classifier.show_most_informative_features(100)